In [0]:
import pandas as pd
import random
import numpy as np
#!pip install nltk
import nltk
#nltk.download()
#!pip install sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from string import punctuation
from nltk.corpus import stopwords
#nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer 
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [0]:
#Loading and Cleaning Data

#load in the csv contatining the data
df_original = pd.read_csv('wiki_movie_plots_deduped.csv.zip')

In [0]:
#drop all rows in the dataframe that do not contain horror(1167) in genre catergory
df_horror=df_original.drop(df_original[df_original['Genre'] != 'horror'].index,inplace=False)
#drop columns we wont need
df_horror.drop(['Release Year','Origin/Ethnicity','Cast', 'Wiki Page','Director','Release Year'],axis=1, inplace = True)

#reduce horror to 1000 rows
df_horror=df_horror.sample(frac=1)
df_horror=df_horror.reset_index(drop=True)
df_horror_train=df_horror[0:1000]
df_horror_test=df_horror[1000:1167]

In [0]:
#drop all rows in dataframe that do not contain comedy(4379)
df_comedy=df_original.drop(df_original[df_original['Genre'] != 'comedy'].index, inplace = False)
df_comedy.drop(['Release Year','Origin/Ethnicity','Cast', 'Wiki Page','Director','Release Year'],axis=1, inplace = True)
#reduce horror to 1000 rows
df_comedy=df_comedy.sample(frac=1)
df_comedy=df_comedy.reset_index(drop=True)
df_comedy_train=df_comedy[0:1000]
df_comedy_test=df_comedy[1000:len(df_comedy)]

In [0]:
#merge the comedy and horror dataframes into one
df_HandC=pd.concat([df_horror_test,df_comedy_test])

In [0]:
def normalizeText(plotSummary):
  normalized_plotSummary = []
    
  for word in plotSummary:
		# remove accented characters
    word= remove_accented_chars(word)
		# expand contractions  (I'm -> I am)  
    word = expand_contractions(word)
        # lowercase the text  (We -> we)  
    word = word.lower()
        # lemmatize text (ran/running is same as run)
    word = nltk.lemmatize_text(word)

         # remove special characters and\or digits   
         # insert spaces between special characters to isolate them    
    special_char_pattern = re.compile(r'([{.(-)!}])')
    word = special_char_pattern.sub(" \\1 ", word)
        # remove digits
    word = nltk.remove_special_characters(word, remove_digits=remove_digits)  
        
        # remove extra whitespace
    word = re.sub(' +', ' ', word)
        # remove stopwords (words like the/a/for have no sentiment)
    word = nltk.remove_stopwords(word, is_lower_case=text_lower_case)

    normalized_plotSummary.append(word)

  return normalized_plotSummary



In [0]:
# run normalizeText on df_HandC
#for row in df_HandC['Plot']:
#  row = normalizeText(row)
  #print(row)
#df_HandC

In [0]:
##Prabesh being stupid. might be helpful?

vectorizer = TfidfVectorizer()
tfidf = vectorizer.fit_transform(df_HandC['Plot'])
#print similarity matrix
print((tfidf*tfidf.T).A)


[[1.         0.09621556 0.09798233 ... 0.10929161 0.06588239 0.06927695]
 [0.09621556 1.         0.14601356 ... 0.11971793 0.08860325 0.09084843]
 [0.09798233 0.14601356 1.         ... 0.14489609 0.10433295 0.10066875]
 ...
 [0.10929161 0.11971793 0.14489609 ... 1.         0.09237811 0.08509669]
 [0.06588239 0.08860325 0.10433295 ... 0.09237811 1.         0.07274998]
 [0.06927695 0.09084843 0.10066875 ... 0.08509669 0.07274998 1.        ]]


In [0]:
##Prabesh being stupid. might be helpful?

#deleting the punctuation
without_punct = [w for w in df_HandC['Plot'] if w not in punctuation]
#deleting stopwords like the/we/for
sw = stopwords.words('english')
without_sw = [w for w in without_punct if w not in sw]
#lemmatizing the plot summary
wnl = WordNetLemmatizer()
lemmatizedSummary = wnl.lemmatize(without_sw)



TypeError: ignored